In [26]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4' # nvidia-smi로 비어있는 gpu 확인하고 여기서 선택할것!

import re

from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
)


from asset.tokenization_kobert import KoBertTokenizer
from asset import tokenization_kobert
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

#from huggingface_hub import notebook_login

#notebook_login()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [27]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [28]:
model = EncoderDecoderModel.from_pretrained('./save_points/dacon-v5/checkpoint-1104000')
model.eval()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

# dacon-v4부터 augmentation 적용됨

In [29]:
import numpy as np
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2
import pandas as pd

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
smodel = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [30]:
# 문장 예시
preds = [
    "이번 경진대회는 질의 응답 처리를 수행하는 AI 모델을 개발해야합니다.",
    "데이콘은 플랫폼입니다."
]

gts = [
    "이번 경진대회의 주제는 도배 하자 질의 응답 AI 모델 개발입니다.",
    "데이콘은 국내 최대의 AI 경진대회 플랫폼입니다."
]

In [31]:
# 샘플에 대한 Cosine Similarity 산식
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [32]:
sample_scores = []
for pred, gt in zip(preds, gts):
    # 생성된 답변 내용을 512 Embedding Vector로 변환
    pred_embed = smodel.encode(pred)
    gt_embed = smodel.encode(gt)
    
    sample_score = cosine_similarity(gt_embed, pred_embed)
    # Cosine Similarity Score가 0보다 작으면 0으로 간주
    sample_score = max(sample_score, 0)
    print('예측 : ', pred)
    print('정답 : ', gt)
    print('Cosine Similarity Score : ', sample_score)
    print('-'*20)
    sample_scores.append(sample_score)
print('전체 샘플의 Cosine Similarity Score 평균 : ', np.mean(sample_scores))

예측 :  이번 경진대회는 질의 응답 처리를 수행하는 AI 모델을 개발해야합니다.
정답 :  이번 경진대회의 주제는 도배 하자 질의 응답 AI 모델 개발입니다.
Cosine Similarity Score :  0.83436465
--------------------
예측 :  데이콘은 플랫폼입니다.
정답 :  데이콘은 국내 최대의 AI 경진대회 플랫폼입니다.
Cosine Similarity Score :  0.6281873
--------------------
전체 샘플의 Cosine Similarity Score 평균 :  0.731276


In [33]:
input_list = []


data = pd.read_csv("./bigdata/train.csv")
input_dict = {"utterance":[]}
for _, row in data.iterrows():
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            input_list.append({"question":row[q_col],"answer":row[a_col]})
data = pd.read_csv('./bigdata/augs_preprocessing/qa_aug.csv')
for _, row in data.iterrows():
    input_list.append({"question":row['Question'],"answer":row['Answer']})
            


In [34]:
def read_input(path):
    test_question = []
    test = pd.read_csv(path)
    for ut in test['질문']:
        test_question.append(ut)

    return test_question

In [35]:
test_q= read_input('./bigdata/test.csv')

In [36]:
gt_q_embed = []

for gt_q_dic in input_list:
    gt_q_embed.append({"question_embed":smodel.encode(gt_q_dic["question"]),"question":gt_q_dic["question"],"answer":gt_q_dic["answer"]})
    


In [37]:
# def trim_incomplete_sentence(text):
#     # 문장이 기호로 끝나는 경우, 이미 완전한 문장임
#     if re.search(r'[\.\?!]$', text):
#         return text
    
#     # . ? ! 과 공백을 찾는 정규 표현식 패턴
#     pattern = re.compile(r'[\.\?!]\s')
    
#     # 모든 매칭을 찾아 리스트로 변환
#     matches = list(pattern.finditer(text))
    
#     if matches:
#         # 가장 마지막 매치의 끝 위치를 찾습니다.
#         last_match = matches[-1]
#         cut_point = last_match.end()
#         return text[:cut_point]
#     else:
#         # 문장 종료 기호가 없다면, 원본 문자열을 그대로 반환합니다.
#         return text

In [38]:
def translate(text,q_penalty=0):
    #print(text)
    max_sim_dic = {"question_embed":None,"question":None,"answer":None}
    max_sim = -100
    text_embed = smodel.encode(text)
    for gt_dic in gt_q_embed:
        tmp_sim = cosine_similarity(gt_dic["question_embed"], text_embed)
        if tmp_sim > max_sim:
            if "장점" or "단점" in text:
                if ("장점" in text and "단점" in gt_dic["question"]) or ("단점" in text and "장점" in gt_dic["question"]):
                    #print("there was an error")
                    continue
            max_sim = tmp_sim
            max_sim_dic = {"question_embed":gt,"question":gt_dic["question"],"answer":gt_dic["answer"]}
    
    
    if max_sim < 0.9:
        
        embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
        embeddings = {k: v for k, v in embeddings.items()}
        # generate 함수에 하이퍼파라미터 추가
        output_sequences = model.generate(
            **embeddings,
            max_length=65-q_penalty,
            temperature=0.9,
            top_k=1,
            top_p=0.9,
            repetition_penalty=1.2,
            do_sample=True,
            num_return_sequences=1
        )
        tmp = trg_tokenizer.decode(output_sequences[0, 1:-1].cpu())
        return tmp

        # dot postprocessing
#         output_sequences = model.generate(
#             **embeddings,
#             max_length=65,
#             temperature=0.9,
#             top_k=1,
#             top_p=0.9,
#             repetition_penalty=1.2,
#             do_sample=True,
#             num_return_sequences=1
#         )
        
    
    
        
    
#         tmp = trg_tokenizer.decode(output_sequences[0, 1:-1].cpu())
        
#         last_period_index = tmp.rfind('.')
        
#         sliced_text = ""
        
#         if last_period_index != -1:  # 마침표가 문자열에 존재하는 경우
#             sliced_text = tmp[:last_period_index + 1]
#         else:  # 마침표가 없는 경우
#             sliced_text = tmp
        
#         return sliced_text
    
    #print("max_sim is...  " +str(max_sim))
    return max_sim_dic["answer"]

In [39]:
import re
def translate_step1(text,q_penalty=0):
    pred = ""
    if '며,' in text:
        # 문장을 '며,'로 분리
        parts = text.split('며,')

        # 첫 번째 부분 출력 (항상 출력)
        pred += translate(parts[0].strip()+'나요?',q_penalty+15)
        
        if len(parts) > 1 and parts[1].strip().startswith('이'):
            # 두 번째 부분이 '이'로 시작하는 경우

            # 앞 문장에서 조사 앞의 단어를 찾기 (가장 마지막에 나오는 조사 앞의 단어)
            match = re.search(r'(.+)(은|는|이|가)(?=\s|$)', parts[0])

            if match:
                # 검색된 단어를 찾아서 '이'를 대체
                subject_word = match.group(1)  # 조사 앞에 있는 스트링
                # '이'를 제거한 나머지 문자열
                replacement = parts[1].strip()[1:]  
                # 단어와 나머지 문자열을 결합
                modified_part = f"{subject_word}{replacement}"  

                # 수정된 두 번째 부분 출력
                pred += " "+translate(modified_part,q_penalty+15)
                
            else:
                # 조사 앞의 단어를 찾지 못한 경우, 원본 두 번째 부분 출력
                pred += " "+translate(parts[1].strip(),q_penalty+15)
        elif len(parts) > 1 and "각각의" in parts[1]:
            # 앞 문장에서 '와', '과'로 연결된 단어 찾기
            match = re.search(r'(\w+)(와|과|나|랑) (\w+)', parts[0])

            if match:
                # '와', '과'가 포함된 단어를 기준으로 새 문장 생성
                word1 = match.group(1)  # 첫 번째 단어
                word2 = match.group(3)  # 두 번째 단어

                # '각각의'를 대체하는 두 문장 생성
                new_sentence1 = parts[1].replace("각각의", word1 + "의", 1)
                new_sentence2 = parts[1].replace("각각의", word2[:-1] + "의", 1)

                pred += " "+translate(new_sentence1.strip(),q_penalty+15)
                pred += " "+translate(new_sentence2.strip(),q_penalty+15)
            else:
                # '와', '과'를 포함하는 패턴을 찾지 못한 경우
                pass

        else:
            # 두 번째 부분이 '이'로 시작하지 않는 경우 또는 없는 경우
            # 조건에 맞는 두 번째 부분이 있다면 출력
            if len(parts) > 1:
                pred += " "+translate(parts[1].strip(),q_penalty+15)


    elif '각각의' in text:
        # '각각의' 앞부분(첫 번째 문장)과 '각각의'를 포함한 뒷부분 분리
        parts = text.split("각각의")
        pred += translate(parts[0].strip(),q_penalty+15)
        # '와', '과'를 기준으로 앞 문장에서 대상을 찾기
        match = re.search(r'(\w+)(와|과|나|랑) (\w+)', parts[0])

        if match:
            # '와', '과'가 포함된 단어를 기준으로 새 문장 생성
            word1 = match.group(1)  # 첫 번째 단어
            word2 = match.group(3)[:-1]  # 두 번째 단어

            # '각각의'를 대체하는 두 문장 생성
            new_sentence1 = word1+"의"+parts[1]
            new_sentence2 = word2+"의"+parts[1]


            pred += " "+translate(new_sentence1.strip(),q_penalty+15)
            pred += " "+translate(new_sentence2.strip(),q_penalty+15)
        else:
            # '와', '과'를 포함하는 패턴을 찾지 못한 경우
            pred += " "+translate('각각의'+parts[1].strip(),q_penalty+15)
            
    else:        
        pred = translate(text,q_penalty)
        
        
    return pred
    

In [40]:
def translate_step0(text):
    pred = ""
    dividers = ["또한,","그리고"]# 며,    . 
    special_divider1 = ["와 ","관계는 무엇인가요?"]
    for divider in dividers:
        
        if divider in text:
            text1 = text.split(divider)[0].strip()
            text2 = text.split(divider)[1].strip()
            pred = translate_step1(text1,15) +" "+divider+" "+ translate_step1(text2,15)
            return pred
    
    
    
    if special_divider1[1] in text:
        text1 = text.split(special_divider1[0])[0].strip()
        text2 = text.split(special_divider1[0])[1].strip()[:-12]
        pred = translate_step1(text1,15) +" "+divider+" "+ translate_step1(text2,15)
        return pred
    
    pred = translate_step1(text)
    return pred
    

In [41]:
# def translate_framework(text):
#     dividers = ["또한,", "그리고"]  # 일반 구분자
#     special_divider = ("와 ", "관계는 무엇인가요?")  # 특별 구분자

#     # 특별 구분자에 대해 텍스트를 먼저 검사하고 처리합니다.
#     if special_divider[1] in text:
#         parts = text.split(special_divider[1])
#         for i, part in enumerate(parts[:-1]):  # 마지막 부분 제외
#             if special_divider[0] in part:
#                 subparts = part.rsplit(special_divider[0], 1)  # 마지막 '와 '로 나눕니다.
#                 parts[i] = translate(subparts[0]) + special_divider[0] + translate(subparts[1])
#             else:
#                 parts[i] = translate(part)
#         text = special_divider[1].join(parts)

#     # 일반 구분자에 대한 처리
#     # 모든 구분자를 하나의 패턴으로 결합합니다.
#     pattern = '|'.join(map(re.escape, dividers))
#     pieces = re.split(pattern, text)
#     translations = [translate(piece) for piece in pieces]

#     # 구분자를 기준으로 원본 텍스트에서 구분자들을 추출합니다.
#     divider_matches = re.findall(pattern, text)

#     # 번역된 부분과 구분자를 다시 조합합니다.
#     result = translations[0]
#     for divider, translation in zip(divider_matches, translations[1:]):
#         result += divider + translation

#     return result

In [42]:
preds = []
for i in range(len(test_q)):
    sr_text = test_q[i] # dialect
    tg_text = translate_step0(sr_text) # translate
    preds.append(tg_text)
    if i>30 and i<60:
        print("source_text : ",sr_text)
        print("result :      ",tg_text)
        print()
        print()

/home/leadawon5/gitfiles/venvs/myvenv/lib/python3.7/site-packages/transformers/generation/utils.py:1260: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"


source_text :  몰딩 수정을 예방하기 위해 건물 내부에서 어떤 종류의 환경 관리가 필요한가요?
result :       도배지에 나타나는 곰팡이의 색깔은 검정색, 황갈색, 분홍색 등이 있습니다. 일반적으로 습기가 있는 환경에서 생긴 검은색 곰팡이가 가장 흔하며, 황갈색과 분홍색 곰팡이는 다양한 환경에서 발견될 수 있습니다. 하지만 곰팡이 발생의 근본 원인인 습도 관리에 주의하고, 사용 시 건강에 무해하지 않은 도구나 농도를 찾는 것이 중요합니다. 알려져


source_text :  KMEW 세라믹 지붕재의 단점에 대해 알려주세요. 또한, 세라믹 타일을 사용할 때 고려해야 할 단점은 무엇인가요?
result :       도배지에 나타나는 곰팡이의 색깔은 검정색, 황갈색, 분홍색 등이 있습니다. 일반적으로 습기가 있는 환경에서 생긴 검은색 곰팡이가 가장 흔하며, 황갈색과 분홍색 곰팡이는 다양한 환경에서 발견될 수 있습니다. 하지만 곰팡이 발생의 근본 원인인 습도 관리에 주의 또한, 세라믹 타일을 사용할 때는 온도 변화에 민감하며, 내장용으로만 적합하고, 경도가 약해 충격과 마모에 취약하다는 주요 단점을 고려해야 하며, 특히 바닥이나 벽면 사용 시 미끄러울 수 있어 주의가 필요합니다.


source_text :  줄퍼티 마감은 무엇인가요? 또한, 액체방수공사는 무엇을 하는 것인가요?
result :       줄퍼티 마감은 석고보드의 이음메 부분을 메꾼 후 1차, 2차로 퍼티를 해 마감하는 것을 의미합니다. 줄퍼티 마감은 올퍼티 마감보다 단가가 저렴하다는 장점이 있지만 이음메 부분의 배부름 현상이 있으며, 줄퍼티를 한 곳과 하지 않은 곳의 질감 차이가 있다는 단점이 있어 가게, 식당, 공장에 주로 시공합니다. 또한, 액체방수공사란 콘크리트, 모르타르 등의 표면에 액체 형태의 방수제를 도포하거나 침투시키고 방수제를 혼합한 모르타르를 덧발라 침투를 막는 공법입니다.


source_text :  페인트 하도재 없이 페인트를 바로 칠할 경우 어떤 문제가

In [43]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

(130, 512)

In [44]:
submit = pd.read_csv('./bigdata/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.023211,-0.019366,0.008139,-0.026836,0.108872,0.011115,0.012168,0.006616,0.004733,...,-0.008703,-0.025681,-0.008075,0.033941,-0.032007,0.072753,0.029727,0.034896,0.012032,0.000082
1,TEST_001,0.029405,-0.002085,-0.003107,-0.023996,0.109395,0.014762,0.031748,0.011764,0.016834,...,-0.024679,0.003839,0.016834,0.035895,-0.047455,0.066697,0.062018,0.006648,0.027433,0.016926
2,TEST_002,0.015530,-0.040067,0.002064,-0.009510,0.097550,0.009840,0.017330,0.006981,0.020562,...,-0.008327,-0.020598,0.007307,0.027334,-0.018009,0.063634,0.014353,0.024675,0.018757,0.018257
3,TEST_003,0.029405,-0.002085,-0.003107,-0.023996,0.109395,0.014762,0.031748,0.011764,0.016834,...,-0.024679,0.003839,0.016834,0.035895,-0.047455,0.066697,0.062018,0.006648,0.027433,0.016926
4,TEST_004,0.029405,-0.002085,-0.003107,-0.023996,0.109395,0.014762,0.031748,0.011764,0.016834,...,-0.024679,0.003839,0.016834,0.035895,-0.047455,0.066697,0.062018,0.006648,0.027433,0.016926


In [45]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('./submit/dacon-v0_submit.csv', index=False)